<a href="https://colab.research.google.com/github/yubingsu/PJ_AI-Beauty/blob/main/flow_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. 개관

## 파일 열기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df_japan = pd.read_excel('./oliveyoung_data/oliveyoung_일본_20251221_141513.xlsx')
df_usa = pd.read_excel('./oliveyoung_data/oliveyoung_미국_20251221_045252.xlsx')
df_china = pd.read_excel('./oliveyoung_data/oliveyoung_중국_20251221_174445.xlsx')

## Download

In [ ]:
!pip install langdetect transformers keybert sentence-transformers scikit-learn

ERROR: Could not install packages due to an OSError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'C:\\Users\\Public\\Documents\\ESTsoft\\CreatorTemp\\pip-unpack-mttqbgjk\\transformers-4.57.3-py3-none-any.whl'
Consider using the `--user` option or check the permissions.




     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------- ----------------------------- 262.1/981.5 kB ? eta -:--:--
     ---------- ----------------------------- 262.1/981.5 kB ? eta -:--:--
     ---------- ----------------------------- 262.1/981.5 kB ? eta -:--:--
     ---------- ----------------------------- 262.1/981.5 kB ? eta -:--:--
     ---------- ----------------------------- 262.1/981.5 kB ? eta -:--:--
     ---------- ----------------------------- 262.1/981.5 kB ? eta -:--:--
     ---------- ----------------------------- 262.1/981.5 kB ? eta -:--:--
     ---------- ----------------------------- 262.1/981.5 kB ? eta -:--:--
     ---------- ----------------------------- 262.1/981.5 kB ? eta -:--:--
     ---------- ----------------

In [ ]:
!pip install transformers langdetect keybert

  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached keybert-0.9.0-py3-none-any.whl.metadata (15 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl.metadata (4.2 kB)
  Using cached sentence_transformers-5.2.0-py3-none-any.whl.metadata (16 kB)
  Using cached torch-2.9.1-cp313-cp313-win_amd64.whl.metadata (30 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--

  DEPRECATION: Building 'langdetect' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'langdetect'. Discussion can be found at https://github.com/pypa/pip/issues/6334


# 2. Pipeline

### [파이프라인 흐름 요약]
1. **데이터 로드** → 엑셀 파일에서 리뷰 데이터 읽기
2. **번역** → 다국어 리뷰를 영어로 번역 (langdetect + NLLB)
3. **문장 분리** → 리뷰를 문장 단위로 분리
4. **감정 분석** → 각 문장에 1~5점 감정 점수 부여, 전체 리뷰 감정 부여
5. **키워드 추출** → KeyBERT로 리뷰당 5개 키워드 추출
6. **클러스터링** → 비슷한 리뷰끼리 그룹화 (0, 1, 2)
7. **결과 출력 및 저장** → 콘솔 출력 + 엑셀 저장

### [결과 해석]
- `ID`: 리뷰 번호
- `cluster`: 비슷한 리뷰 그룹 (같은 번호 = 유사한 내용)
- `score`: 문장 감정 (1=매우부정, 5=매우긍정)
- `Review sentiment`: 전체 리뷰 감정 (positive/neutral/negative)
- `KeyBERT keywords`: 추출된 핵심 키워드

In [ ]:
# =============================================================================
# 올리브영 리뷰 분석 파이프라인 - 리뷰 키워드 추출 (테스트용 10개)
# =============================================================================

import pandas as pd
import numpy as np
from transformers import pipeline
from langdetect import detect
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import re
from collections import defaultdict, Counter
from pprint import pprint

# -----------------------------------------------------------------------------
# 1. 모델 초기화
# -----------------------------------------------------------------------------

translator = pipeline("translation", model="facebook/nllb-200-distilled-600M")
kw_model = KeyBERT()

lang_map = {
    'ja': 'jpn_Jpan',
    'ko': 'kor_Hang',
    'en': 'eng_Latn',
    'zh-cn': 'zho_Hans',
}


Device set to use cpu


In [ ]:

# -----------------------------------------------------------------------------
# 2. 번역 함수
# -----------------------------------------------------------------------------

def normalize_language_and_translate(text):
    try:
        if pd.isna(text) or str(text).strip() == '':
            return {'lang': 'unknown', 'text_en': ''}

        lang_code = detect(str(text))

        if lang_code == 'en':
            return {'lang': 'eng_Latn', 'text_en': str(text)}

        src_lang = lang_map.get(lang_code, 'jpn_Jpan')
        translated = translator(
            str(text),
            src_lang=src_lang,
            tgt_lang='eng_Latn',
            max_length=512
        )[0]['translation_text']

        return {'lang': src_lang, 'text_en': translated}
    except:
        return {'lang': 'unknown', 'text_en': str(text)}

# -----------------------------------------------------------------------------
# 3. 문장 분리 함수
# -----------------------------------------------------------------------------

def split_sentences(text):
    if not text:
        return []
    sentences = re.split(r'[.!?]+', text)
    return [s.strip() for s in sentences if s.strip()]

# -----------------------------------------------------------------------------
# 4. 문장별 감정 점수 (임의)
# -----------------------------------------------------------------------------
def sentiment_of_sentence(sentence):
    """
    문장의 감정 점수 반환 (1~5)
    1: 매우 부정, 2: 부정, 3: 중립, 4: 긍정, 5: 매우 긍정
    """
    if not sentence or len(sentence.strip()) < 3:
        return 3

    try:
        out = sentiment_pipe(sentence)[0]
        return int(out["label"][0])
    except:
        return 3

# -----------------------------------------------------------------------------
# 5. 리뷰 문장 전체 감정 판단 (임의)
# -----------------------------------------------------------------------------

def sentiment_of_review(sentences):
    """
    리뷰 전체 감정 판단
    """
    if not sentences:
        return "neutral"

    scores = [sentiment_of_sentence(s) for s in sentences]

    avg_score = sum(scores) / len(scores)
    neg_ratio = sum(1 for s in scores if s <= 2) / len(scores)

    if neg_ratio >= 0.3:
        return "negative"

    if avg_score >= 4.0:
        return "positive"
    elif avg_score >= 3.0:
        return "neutral"
    else:
        return "negative"

# -----------------------------------------------------------------------------
# 6. KeyBERT 키워드 추출
# -----------------------------------------------------------------------------

custom_stop_words = "english"
    #+ [ #불용어 설정
    #'very', 'truly', 'really', 'quite', 'so',
    #'just', 'absolutely', 'totally', 'extremely' ]

def extract_keywords_keybert(text, top_n=5):
    if not text or len(text.strip()) < 10:
        return []

    try:
        keywords = kw_model.extract_keywords(
            text,
            keyphrase_ngram_range=(1, 2),  # 1-2개 단어 조합 추출 (예: "second purchase", "absorbs very quickly")
            stop_words=custom_stop_words,   # 불용어 제거 (의미 없는 단어 필터링)
            top_n=top_n,                    # 상위 N개 키워드만 추출
            use_mmr=True,                   # MMR 알고리즘 사용 (다양성 확보, 키워드 중복 제거)
            diversity=0.5                   # 0.5 = 관련성과 다양성의 균형 (0.0-1.0, 높을수록 서로 다른 키워드 추출)
        )
        return [kw[0] for kw in keywords]
    except:
        return []

# -----------------------------------------------------------------------------
# 7. 제품별 리뷰 파이프라인 [최종]
# -----------------------------------------------------------------------------

def run_review_pipeline(df, top_n_reviews=None):
    """
    제품별 리뷰 분석 파이프라인

    Args:
        df: 입력 데이터프레임
        top_n_reviews: 처리할 리뷰 개수 (기본값None: 전체)

    Returns:
        list: 처리된 리뷰 결과
    """

    # None이면 전체, 아니면 지정된 개수
    if top_n_reviews is None:
        df_sample = df.copy()
    else:
        df_sample = df.head(top_n_reviews).copy()

    processed = []
    total_reviews = len(df_sample)

    # [개발/테스트용 - 자동화 시 제거 가능] 처리 시작 메시지
    print("===== PROCESSING REVIEWS =====")
    print(f"총 {total_reviews}개 리뷰 처리 시작\n")

    for idx, row in df_sample.iterrows():
        # [개발/테스트용 - 자동화 시 제거 가능] 50개마다 진행 상황 출력
        current = idx + 1
        if current % 50 == 0 or current == total_reviews:
            print(f"진행중: {current}/{total_reviews} 리뷰 처리 완료")

        review_id = f"r{idx+1}"
        country = row.get('country', 'Unknown')
        platform = 'oliveyoung_global'
        review_text = row.get('review_content', '')

        norm = normalize_language_and_translate(review_text)
        sentences = split_sentences(norm['text_en'])

        sentence_records = []
        for s in sentences:
            score = sentiment_of_sentence(s)
            sentence_records.append({'sentence': s, 'score': score})

        review_sentiment = sentiment_of_review(sentences)
        keywords = extract_keywords_keybert(norm['text_en'], top_n=5)

        processed.append({
            'id': review_id,
            'country': country,
            'platform': platform,
            'major_category': row.get('major_category', ''),
            'sub_category': row.get('sub_category', ''),
            'product_name': row.get('product_name', ''),
            'brand': row.get('brand', ''),
            'lang': norm['lang'],
            'text_en': norm['text_en'],
            'sentences': sentence_records,
            'review_sentiment': review_sentiment,
            'kw_keybert': keywords,
            'cluster': 0
        })

    # [개발/테스트용 - 자동화 시 제거 가능] 리뷰 처리 완료 메시지
    print(f"\n리뷰 처리 완료: {total_reviews}개")

    if len(processed) >= 3:
        # [개발/테스트용 - 자동화 시 제거 가능] 클러스터링 시작 메시지
        print("클러스터링 시작...")

        corpus = [p['text_en'] for p in processed]
        vectorizer = TfidfVectorizer(max_df=0.9, min_df=1, ngram_range=(1, 2), stop_words='english')
        X = vectorizer.fit_transform(corpus)
        n_clusters = max(1, min(3, len(corpus)))
        km = KMeans(n_clusters=n_clusters, random_state=42)
        km.fit(X)
        labels = km.labels_

        for i, p in enumerate(processed):
            p['cluster'] = int(labels[i])

        # [개발/테스트용 - 자동화 시 제거 가능] 클러스터링 완료 메시지
        print("클러스터링 완료")

    return processed

# -----------------------------------------------------------------------------
# 8. 결과 출력 함수
# -----------------------------------------------------------------------------

def print_processed_reviews(processed, show_sample=True, sample_size=20):
    """
    처리된 리뷰 결과 출력

    Args:
        processed: 처리된 리뷰 리스트
        show_sample: 샘플 출력 여부
        sample_size: 출력할 샘플 개수
    """
    print('\n===== 분석 결과 요약 =====')
    print(f"총 처리된 리뷰: {len(processed)}개")

    # 감정 분포
    sentiments = [p.get('review_sentiment') for p in processed]
    positive = sentiments.count('positive')
    neutral = sentiments.count('neutral')
    negative = sentiments.count('negative')

    print(f"긍정: {positive}개, 중립: {neutral}개, 부정: {negative}개")

    # 클러스터 분포
    clusters = [p.get('cluster') for p in processed]
    cluster_counts = {i: clusters.count(i) for i in set(clusters)}
    print(f"클러스터 분포: {cluster_counts}")

    # 샘플 출력
    if show_sample:
        print(f'\n===== 샘플 리뷰 (처음 {min(sample_size, len(processed))}개) =====')
        for p in processed[:sample_size]:
            print(f"\nID: {p['id']} | country: {p['country']} | platform: {p['platform']} | lang: {p['lang']} | cluster: {p['cluster']}")
            for s in p['sentences']:
                print(' -', s)
            print(' Review sentiment:', p.get('review_sentiment'))
            print(' KeyBERT keywords:', p.get('kw_keybert'))

# -----------------------------------------------------------------------------
# 9. 결과를 테이블로 저장 - 확인/기획용
# -----------------------------------------------------------------------------

from datetime import datetime

def save_results_to_table(processed, output_file=None, country=None):
    """
    처리된 리뷰 결과를 테이블 형태로 저장

    Args:
        processed: 처리된 리뷰 리스트
        output_file: 출력 파일명 (None이면 자동 생성)
        country: 국가명 (파일명과 메타데이터용)
    """

    # 국가명 자동 추출 (processed에서)
    if country is None and len(processed) > 0:
        country = processed[0].get('country', 'Unknown')

    # 타임스탬프 생성
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    # 파일명 자동 생성
    if output_file is None:
        output_file = f'./oliveyoung_data/review_analysis_results_{country}_{timestamp}.xlsx'

    rows = []

    for p in processed:
        sentences_text = ' | '.join([f"{s['sentence']} (score: {s['score']})" for s in p['sentences']])

        result_row = {
            'review_id': p['id'],
            'country': p['country'],
            'platform': p['platform'],
            'major_category': p.get('major_category', ''),
            'sub_category': p.get('sub_category', ''),
            'product_name': p.get('product_name', ''),
            'brand': p.get('brand', ''),
            'language': p['lang'],
            'translated_text': p['text_en'],
            'sentences': sentences_text,
            'review_sentiment': p['review_sentiment'],
            'keywords': ', '.join(p['kw_keybert']),
            'cluster': p['cluster'],
            'analysis_timestamp': timestamp
        }
        rows.append(result_row)

    result_df = pd.DataFrame(rows)
    result_df.to_excel(output_file, index=False)

    # 출력 메시지에 국가와 시간 포함
    print(f"\n[{country}] 분석 완료: {timestamp}")
    print(f"결과가 {output_file}에 저장되었습니다")
    print(f"총 {len(processed)}개 리뷰 처리")

    return result_df

In [ ]:
# 최종적으로는 상품별로 뽑는 것을 고려해야하는 것이 아닌지?

In [ ]:
# =============================================================================
# 실행
# =============================================================================

if __name__ == "__main__":
    # 데이터 로드
    df = pd.read_excel('./oliveyoung_data/oliveyoung_미국_20251221_045252.xlsx')

    # 파이프라인 실행
    processed = run_review_pipeline(df, top_n_reviews=len(df))

    # 결과 출력 [개발/테스트용 - 자동화 시 제거 가능]
    print_processed_reviews(processed)

    # 결과 저장 (자동으로 국가명+타임스탬프 파일명 생성) [개발/테스트용 - 자동화 시 제거 가능]
    result_df = save_results_to_table(processed)


===== PROCESSING REVIEWS =====
총 12025개 리뷰 처리 시작

진행중: 50/12025 리뷰 처리 완료
진행중: 100/12025 리뷰 처리 완료
진행중: 150/12025 리뷰 처리 완료
진행중: 200/12025 리뷰 처리 완료
진행중: 250/12025 리뷰 처리 완료
진행중: 300/12025 리뷰 처리 완료
진행중: 350/12025 리뷰 처리 완료
진행중: 400/12025 리뷰 처리 완료
진행중: 450/12025 리뷰 처리 완료
진행중: 500/12025 리뷰 처리 완료
진행중: 550/12025 리뷰 처리 완료
진행중: 600/12025 리뷰 처리 완료
진행중: 650/12025 리뷰 처리 완료
진행중: 700/12025 리뷰 처리 완료
진행중: 750/12025 리뷰 처리 완료
진행중: 800/12025 리뷰 처리 완료
진행중: 850/12025 리뷰 처리 완료
진행중: 900/12025 리뷰 처리 완료
진행중: 950/12025 리뷰 처리 완료
진행중: 1000/12025 리뷰 처리 완료
진행중: 1050/12025 리뷰 처리 완료
진행중: 1100/12025 리뷰 처리 완료
진행중: 1150/12025 리뷰 처리 완료
진행중: 1200/12025 리뷰 처리 완료
진행중: 1250/12025 리뷰 처리 완료
진행중: 1300/12025 리뷰 처리 완료
진행중: 1350/12025 리뷰 처리 완료
진행중: 1400/12025 리뷰 처리 완료
진행중: 1450/12025 리뷰 처리 완료
진행중: 1500/12025 리뷰 처리 완료
진행중: 1550/12025 리뷰 처리 완료
진행중: 1600/12025 리뷰 처리 완료
진행중: 1650/12025 리뷰 처리 완료
진행중: 1700/12025 리뷰 처리 완료
진행중: 1750/12025 리뷰 처리 완료
진행중: 1800/12025 리뷰 처리 완료
진행중: 1850/12025 리뷰 처리 완료
진행중: 1900/12025 리뷰 처리 완료
진행중: 1950/12025 리뷰 처

In [ ]:
# 3. K-Beauty 제품 속성 추출 파이프라인
# Claude API를 사용하여 화장품 제품 정보에서 구조화된 속성 자동 추출